In [1]:
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt
import glob
import os
from goes_ortho import ortho

**Download radiance product (ABI-L1b-RadC) for ABI channels 1-16 (C01- C16):**

`python ./download-goes.py --bucket noaa-goes16 --year 2020 --month 2 --days 7 9 --product ABI-L1b-RadC --channel C01 --bounds 30 50 -125 -100 --dir /storage/GOES`

**Download LST product:**

`python ./download-goes.py --bucket noaa-goes16 --year 2020 --month 2 --days 7 9 --product ABI-L2-LSTC --channel none --bounds 30 50 -125 -100 --dir /storage/GOES`

---

In [2]:
def batch_ortho(filepath_list, destination_path, variable):
    # Orthorectify each GOES ABI LST image
    for i, goes_image_path in enumerate(filepath_list):
        
        print('Orthorectifying file {} of {}'.format(i+1, len(filepath_list)))
       
        new_goes_image_path = destination_path + goes_image_path.split('/')[-1].split('.')[0] + '_orthorectified_GrandMesa.nc'
        try:
            if not os.path.exists(new_goes_image_path):
                ortho(goes_image_path, 
                      data_vars = [variable], 
                      bounds = (-108.4, 38.8, -107.9, 39.2), 
                      api_key = '585b1d1639bc5ef8a4a5bdea7e45a8d1', 
                      new_goes_filename=new_goes_image_path, 
                      dem_filepath=None, 
                      demtype='SRTMGL3', 
                      keep_dem=True)
            else:
                print('File already exists, skipping...')
        except AttributeError as attr_err:
            print(attr_err)
        
    # Convert each orthorectified GOES ABI image to a geotiff
    for i, goes_image_path in enumerate(filepath_list):
        
        new_goes_image_path = destination_path + goes_image_path.split('/')[-1].split('.')[0] + '_orthorectified_GrandMesa.nc'
        
        # only get netcdf files
        if os.path.splitext(os.path.normpath(new_goes_image_path))[-1] == '.nc':
        
            print('Converting file {} of {}'.format(i+1, len(filepath_list)))
            
            # get the orthorectified GOES NetCDF filepath
            netcdf_filepath = os.path.normpath(new_goes_image_path)
            #print(netcdf_filepath)
            
            # create a geotiff for Radiance, Brightness Temperature, and zone_labels for zonal statistics
            for band in [variable]:
                # make a filepath for the new tif file
                geotiff_filepath = '{}_{}.tif'.format(os.path.splitext(netcdf_filepath)[0],band)
                #print(geotiff_filepath)
                
                if not os.path.exists(geotiff_filepath):
                    # using gdal in the command line here to convert
                    os.system('gdal_translate -a_srs {a_srs} -of GTiff NETCDF:{netcdf_filepath}:{band} {geotiff_filepath}'.format(
                        a_srs = 'EPSG:4326',
                        netcdf_filepath = netcdf_filepath,
                        geotiff_filepath = geotiff_filepath,
                        band = band))
                else:
                    print('File already exists, skipping...')

---
**Crop, orthorectify, and convert LST images to GeoTIFFs**

In [3]:
lst_files = glob.glob(r'/storage/GOES/goes16/2020/2/**/ABI-L2-LSTC/**/*.nc',recursive = True)

In [4]:
batch_ortho(filepath_list = lst_files, 
            destination_path = '/storage/GOES/orthorectified/GrandMesa2020/',
            variable = 'LST')

Orthorectifying file 1 of 72
File already exists, skipping...
Orthorectifying file 2 of 72
File already exists, skipping...
Orthorectifying file 3 of 72
File already exists, skipping...
Orthorectifying file 4 of 72
File already exists, skipping...
Orthorectifying file 5 of 72
File already exists, skipping...
Orthorectifying file 6 of 72
File already exists, skipping...
Orthorectifying file 7 of 72
File already exists, skipping...
Orthorectifying file 8 of 72
File already exists, skipping...
Orthorectifying file 9 of 72
File already exists, skipping...
Orthorectifying file 10 of 72
File already exists, skipping...
Orthorectifying file 11 of 72
File already exists, skipping...
Orthorectifying file 12 of 72
File already exists, skipping...
Orthorectifying file 13 of 72
File already exists, skipping...
Orthorectifying file 14 of 72
File already exists, skipping...
Orthorectifying file 15 of 72
File already exists, skipping...
Orthorectifying file 16 of 72
File already exists, skipping...
O

---
**Crop, orthorectify, and convert Radiance images to GeoTIFFs**

In [5]:
rad_files = glob.glob(r'/storage/GOES/goes16/2020/2/**/ABI-L1b-RadC/**/*.nc',recursive = True)

In [6]:
batch_ortho(filepath_list = rad_files, 
            destination_path = '/storage/GOES/orthorectified/GrandMesa2020/',
            variable = 'Rad')

Orthorectifying file 1 of 13826
File already exists, skipping...
Orthorectifying file 2 of 13826
File already exists, skipping...
Orthorectifying file 3 of 13826
File already exists, skipping...
Orthorectifying file 4 of 13826
File already exists, skipping...
Orthorectifying file 5 of 13826
File already exists, skipping...
Orthorectifying file 6 of 13826
File already exists, skipping...
Orthorectifying file 7 of 13826
File already exists, skipping...
Orthorectifying file 8 of 13826
File already exists, skipping...
Orthorectifying file 9 of 13826
File already exists, skipping...
Orthorectifying file 10 of 13826
File already exists, skipping...
Orthorectifying file 11 of 13826
File already exists, skipping...
Orthorectifying file 12 of 13826
File already exists, skipping...
Orthorectifying file 13 of 13826
File already exists, skipping...
Orthorectifying file 14 of 13826
File already exists, skipping...
Orthorectifying file 15 of 13826
File already exists, skipping...
Orthorectifying fil

File already exists, skipping...
Orthorectifying file 742 of 13826
File already exists, skipping...
Orthorectifying file 743 of 13826
File already exists, skipping...
Orthorectifying file 744 of 13826
File already exists, skipping...
Orthorectifying file 745 of 13826
File already exists, skipping...
Orthorectifying file 746 of 13826
File already exists, skipping...
Orthorectifying file 747 of 13826
File already exists, skipping...
Orthorectifying file 748 of 13826
File already exists, skipping...
Orthorectifying file 749 of 13826
File already exists, skipping...
Orthorectifying file 750 of 13826
File already exists, skipping...
Orthorectifying file 751 of 13826
File already exists, skipping...
Orthorectifying file 752 of 13826
File already exists, skipping...
Orthorectifying file 753 of 13826
File already exists, skipping...
Orthorectifying file 754 of 13826
File already exists, skipping...
Orthorectifying file 755 of 13826
File already exists, skipping...
Orthorectifying file 756 of 1

File already exists, skipping...
Orthorectifying file 1595 of 13826
File already exists, skipping...
Orthorectifying file 1596 of 13826
File already exists, skipping...
Orthorectifying file 1597 of 13826
File already exists, skipping...
Orthorectifying file 1598 of 13826
File already exists, skipping...
Orthorectifying file 1599 of 13826
File already exists, skipping...
Orthorectifying file 1600 of 13826
File already exists, skipping...
Orthorectifying file 1601 of 13826
File already exists, skipping...
Orthorectifying file 1602 of 13826
File already exists, skipping...
Orthorectifying file 1603 of 13826
File already exists, skipping...
Orthorectifying file 1604 of 13826
File already exists, skipping...
Orthorectifying file 1605 of 13826
File already exists, skipping...
Orthorectifying file 1606 of 13826
File already exists, skipping...
Orthorectifying file 1607 of 13826
File already exists, skipping...
Orthorectifying file 1608 of 13826
File already exists, skipping...
Orthorectifying

File already exists, skipping...
Orthorectifying file 2500 of 13826
File already exists, skipping...
Orthorectifying file 2501 of 13826
File already exists, skipping...
Orthorectifying file 2502 of 13826
File already exists, skipping...
Orthorectifying file 2503 of 13826
File already exists, skipping...
Orthorectifying file 2504 of 13826
File already exists, skipping...
Orthorectifying file 2505 of 13826
File already exists, skipping...
Orthorectifying file 2506 of 13826
File already exists, skipping...
Orthorectifying file 2507 of 13826
File already exists, skipping...
Orthorectifying file 2508 of 13826
File already exists, skipping...
Orthorectifying file 2509 of 13826
File already exists, skipping...
Orthorectifying file 2510 of 13826
File already exists, skipping...
Orthorectifying file 2511 of 13826
File already exists, skipping...
Orthorectifying file 2512 of 13826
File already exists, skipping...
Orthorectifying file 2513 of 13826
File already exists, skipping...
Orthorectifying

File already exists, skipping...
Orthorectifying file 3369 of 13826
File already exists, skipping...
Orthorectifying file 3370 of 13826
File already exists, skipping...
Orthorectifying file 3371 of 13826
File already exists, skipping...
Orthorectifying file 3372 of 13826
File already exists, skipping...
Orthorectifying file 3373 of 13826
File already exists, skipping...
Orthorectifying file 3374 of 13826
File already exists, skipping...
Orthorectifying file 3375 of 13826
File already exists, skipping...
Orthorectifying file 3376 of 13826
File already exists, skipping...
Orthorectifying file 3377 of 13826
File already exists, skipping...
Orthorectifying file 3378 of 13826
File already exists, skipping...
Orthorectifying file 3379 of 13826
File already exists, skipping...
Orthorectifying file 3380 of 13826
File already exists, skipping...
Orthorectifying file 3381 of 13826
File already exists, skipping...
Orthorectifying file 3382 of 13826
File already exists, skipping...
Orthorectifying

...

File already exists, skipping...
Converting file 9408 of 13826
File already exists, skipping...
Converting file 9409 of 13826
File already exists, skipping...
Converting file 9410 of 13826
File already exists, skipping...
Converting file 9411 of 13826
File already exists, skipping...
Converting file 9412 of 13826
File already exists, skipping...
Converting file 9413 of 13826
File already exists, skipping...
Converting file 9414 of 13826
File already exists, skipping...
Converting file 9415 of 13826
File already exists, skipping...
Converting file 9416 of 13826
File already exists, skipping...
Converting file 9417 of 13826
File already exists, skipping...
Converting file 9418 of 13826
File already exists, skipping...
Converting file 9419 of 13826
File already exists, skipping...
Converting file 9420 of 13826
File already exists, skipping...
Converting file 9421 of 13826
File already exists, skipping...
Converting file 9422 of 13826
File already exists, skipping...
Converting file 9423 o

File already exists, skipping...
Converting file 10259 of 13826
File already exists, skipping...
Converting file 10260 of 13826
File already exists, skipping...
Converting file 10261 of 13826
File already exists, skipping...
Converting file 10262 of 13826
File already exists, skipping...
Converting file 10263 of 13826
File already exists, skipping...
Converting file 10264 of 13826
File already exists, skipping...
Converting file 10265 of 13826
File already exists, skipping...
Converting file 10266 of 13826
File already exists, skipping...
Converting file 10267 of 13826
File already exists, skipping...
Converting file 10268 of 13826
File already exists, skipping...
Converting file 10269 of 13826
File already exists, skipping...
Converting file 10270 of 13826
File already exists, skipping...
Converting file 10271 of 13826
File already exists, skipping...
Converting file 10272 of 13826
File already exists, skipping...
Converting file 10273 of 13826
File already exists, skipping...
Convert

File already exists, skipping...
Converting file 11221 of 13826
File already exists, skipping...
Converting file 11222 of 13826
File already exists, skipping...
Converting file 11223 of 13826
File already exists, skipping...
Converting file 11224 of 13826
File already exists, skipping...
Converting file 11225 of 13826
File already exists, skipping...
Converting file 11226 of 13826
File already exists, skipping...
Converting file 11227 of 13826
File already exists, skipping...
Converting file 11228 of 13826
File already exists, skipping...
Converting file 11229 of 13826
File already exists, skipping...
Converting file 11230 of 13826
File already exists, skipping...
Converting file 11231 of 13826
File already exists, skipping...
Converting file 11232 of 13826
File already exists, skipping...
Converting file 11233 of 13826
File already exists, skipping...
Converting file 11234 of 13826
File already exists, skipping...
Converting file 11235 of 13826
File already exists, skipping...
Convert

File already exists, skipping...
Converting file 12252 of 13826
File already exists, skipping...
Converting file 12253 of 13826
File already exists, skipping...
Converting file 12254 of 13826
File already exists, skipping...
Converting file 12255 of 13826
File already exists, skipping...
Converting file 12256 of 13826
File already exists, skipping...
Converting file 12257 of 13826
File already exists, skipping...
Converting file 12258 of 13826
File already exists, skipping...
Converting file 12259 of 13826
File already exists, skipping...
Converting file 12260 of 13826
File already exists, skipping...
Converting file 12261 of 13826
File already exists, skipping...
Converting file 12262 of 13826
File already exists, skipping...
Converting file 12263 of 13826
File already exists, skipping...
Converting file 12264 of 13826
File already exists, skipping...
Converting file 12265 of 13826
File already exists, skipping...
Converting file 12266 of 13826
File already exists, skipping...
Convert

File already exists, skipping...
Converting file 13079 of 13826
File already exists, skipping...
Converting file 13080 of 13826
File already exists, skipping...
Converting file 13081 of 13826
File already exists, skipping...
Converting file 13082 of 13826
File already exists, skipping...
Converting file 13083 of 13826
File already exists, skipping...
Converting file 13084 of 13826
File already exists, skipping...
Converting file 13085 of 13826
File already exists, skipping...
Converting file 13086 of 13826
File already exists, skipping...
Converting file 13087 of 13826
File already exists, skipping...
Converting file 13088 of 13826
File already exists, skipping...
Converting file 13089 of 13826
File already exists, skipping...
Converting file 13090 of 13826
File already exists, skipping...
Converting file 13091 of 13826
File already exists, skipping...
Converting file 13092 of 13826
File already exists, skipping...
Converting file 13093 of 13826
File already exists, skipping...
Convert